In [1534]:
%load_ext autoreload
#To update the imported files

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1535]:
%autoreload
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import mean_absolute_error
from utilities import *
import seaborn as sns
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [1536]:
#Loading the y-paramter from file, might be changed to y_a etc.
train_a = pd.read_parquet('../Data_and_task/A/train_targets.parquet')
train_b = pd.read_parquet('../Data_and_task/B/train_targets.parquet')
train_c = pd.read_parquet('../Data_and_task/C/train_targets.parquet')

In [1537]:
#Loading estimated/forecasted training_weather from file
X_train_estimated_a = pd.read_parquet('../Data_and_task/A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../Data_and_task/B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../Data_and_task/C/X_train_estimated.parquet')

In [1538]:
#Loading observed weather from file
X_train_observed_a = pd.read_parquet('../Data_and_task/A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('../Data_and_task/B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('../Data_and_task/C/X_train_observed.parquet')

In [1539]:
#Loading estimated/forecasted test_weather from file
X_test_estimated_a = pd.read_parquet('../Data_and_task/A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('../Data_and_task/B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('../Data_and_task/C/X_test_estimated.parquet')

In [1540]:
#Cleaning data such that only wanted features are included

selected_features = ["date_forecast", "direct_rad:W", "direct_rad_1h:J", "clear_sky_rad:W", "clear_sky_energy_1h:J", "diffuse_rad:W",
"diffuse_rad_1h:J", "sun_elevation:d", "is_in_shadow:idx", "is_day:idx"]

#Defining y_fetures, will always be the same
y_features = ["time", "pv_measurement"]

#Fjernen kvarter-radene og fjerner "date_calc" kolonnen hvis den finnes
X_train_observed_a_clean = clean_X_data(X_train_observed_a)
X_train_observed_b_clean = clean_X_data(X_train_observed_b)
X_train_observed_c_clean = clean_X_data(X_train_observed_c)
X_train_estimated_a_clean = clean_X_data(X_train_observed_a)
X_train_estimated_b_clean = clean_X_data(X_train_observed_b)
X_train_estimated_c_clean = clean_X_data(X_train_observed_c)
X_test_estimated_a_clean = clean_X_data(X_test_estimated_a)
X_test_estimated_b_clean = clean_X_data(X_test_estimated_b)
X_test_estimated_c_clean = clean_X_data(X_test_estimated_c)

X_train_observed_a_clean_selected_features = X_train_observed_a_clean[selected_features].copy()
X_train_observed_b_clean_selected_features = X_train_observed_b_clean[selected_features].copy()
X_train_observed_c_clean_selected_features = X_train_observed_c_clean[selected_features].copy()
X_train_estimated_a_clean_selected_features = X_train_estimated_a_clean[selected_features].copy()
X_train_estimated_b_clean_selected_features = X_train_estimated_b_clean[selected_features].copy()
X_train_estimated_c_clean_selected_features = X_train_estimated_c_clean[selected_features].copy()
X_test_estimated_a_clean_selected_features = X_test_estimated_a_clean[selected_features].copy()
X_test_estimated_b_clean_selected_features = X_test_estimated_b_clean[selected_features].copy()
X_test_estimated_c_clean_selected_features = X_test_estimated_c_clean[selected_features].copy()

In [1541]:
"""train_b.drop(train_b.index[335:720], inplace=True)
train_b.drop(train_b.index[10966:11306], inplace=True)
train_b.drop(train_b.index[13411:14469], inplace=True)
train_b.drop(train_b.index[15180:15216], inplace=True)
train_b.drop(train_b.index[18690:20140], inplace=True)
train_b.drop(train_b.index[20370:20441], inplace=True)
train_b.drop(train_b.index[21260:21315], inplace=True)
# Sletter rader:335 til 720, 10966 til 11306, 13411 til 14469, 15180 til 15216, 18690 til 20140, 20370 til 20441, 21260 til 21315"""

'train_b.drop(train_b.index[335:720], inplace=True)\ntrain_b.drop(train_b.index[10966:11306], inplace=True)\ntrain_b.drop(train_b.index[13411:14469], inplace=True)\ntrain_b.drop(train_b.index[15180:15216], inplace=True)\ntrain_b.drop(train_b.index[18690:20140], inplace=True)\ntrain_b.drop(train_b.index[20370:20441], inplace=True)\ntrain_b.drop(train_b.index[21260:21315], inplace=True)\n# Sletter rader:335 til 720, 10966 til 11306, 13411 til 14469, 15180 til 15216, 18690 til 20140, 20370 til 20441, 21260 til 21315'

In [1542]:
train_c.dropna(inplace=True)

# delete_ranges_of_zeros(train_c, 'pv_measurement', 5, [19.6, 9.8])

deleted_ranges = delete_sequence_with_repeating_numbers(train_b, "pv_measurement", 25)
delete_repeating_non_zero_numbers(train_b, 'pv_measurement', 3)

In [1543]:
#Training a Linear regression model on X_observed_a and testing it on X_estimated_a and evaluating it on MAE, PURELY for testing!
#See below for how its done when submitting
X_train_observed_b_clean_selected_features_resized, train_b_observed_resized = resize_trainingdata(X_train_observed_b_clean_selected_features, train_b, "date_forecast", y_features)
X_train_estimated_b_clean_selected_features_resized, train_b_estimated_resized = resize_trainingdata(X_train_estimated_b_clean_selected_features, train_b, "date_forecast", y_features)

#Scaling the data for more fair comparions and faster convergence, ChatGPT
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_observed_b_clean_selected_features_resized)
X_test_scaled = scaler.fit_transform(X_train_estimated_b_clean_selected_features_resized)

#Training the model
reg = LinearRegression()
reg.fit(X_train_scaled, train_b_observed_resized["pv_measurement"])

# Make predictions
y_pred = reg.predict(X_test_scaled)

#Evaluate mean asbolute error
mae = mean_absolute_error(train_b_estimated_resized["pv_measurement"], y_pred)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 13.732492558093316
